# 7. Provisioning an Azure ML Compute cluster  

To run training jobs in the cloud we need an **AMLCompute** target.  
This notebook cell will either **reuse** an existing cluster or **create** a new one with autoscaling rules.

**Key concepts**

| Concept | Why it matters | Default in this demo |
|---------|----------------|----------------------|
| **VM size (`size`)** | Determines CPU/GPU type, RAM & disk. | `"STANDARD_D2_V3"` (CPU) — change to e.g. `"Standard_NC6s_v3"` for a single Tesla V100. |
| **Autoscaling** | Saves cost by shrinking to `min_instances` when idle, up to `max_instances` on demand. | `min=0`, `max=4`, scale-down after **3 min** idle. |
| **Tier** | `Dedicated` (pay-as-you-go) or `LowPriority` (pre-emptible, cheaper). | `Dedicated`. |
| **Idempotent creation** | Running the cell twice will detect the cluster and skip re-creation. | Handled in the helper. |

> **Quota check**  
> Ensure your subscription has quota for the chosen VM size in the selected region.  
> If not, request a quota increase in the Azure portal before running this cell.


In [1]:
import yaml

from azure.identity import DefaultAzureCredential

from azure.ai.ml import MLClient

from azure.ai.ml.entities import AmlCompute

## 1) Load settings from config.yaml

In [9]:
# Load configuration from the YAML file
with open("../config.yaml", "r") as file:
    config = yaml.safe_load(file)

In [10]:
subscription_id = config["azure"]["subscription_id"]
resource_group_name = config["azure"]["resource_group_name"]
workspace_name = config["azure"]["workspace_name"]

training_gpu_cluster = config["azure"]["training_gpu_cluster"]
compute_name = config["azure"]["compute_name"]

## 2) Authenticate & connect to the workspace

In [11]:
# Initialize DefaultAzureCredential
credential = DefaultAzureCredential()

In [12]:
# Update Azure ML Client
ml_client = MLClient(credential, subscription_id, resource_group_name, workspace_name)

Overriding of current TracerProvider is not allowed
Overriding of current LoggerProvider is not allowed
Overriding of current MeterProvider is not allowed
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented


## 3) Create or verify Compute Resource

In [13]:
def create_compute_resource(ml_client, cluster_name, size="STANDARD_D2_V3", 
                            min_instances=0, max_instances=4, 
                            idle_time_before_scale_down=180, tier="Dedicated"):
    """
    Creates or reuses an AMLCompute resource in Azure ML.

    Parameters:
        ml_client (MLClient): The Azure ML client.
        cluster_name (str): The name of the compute cluster.
        size (str): The VM size (default is "STANDARD_D2_V3").
        min_instances (int): The minimum number of instances (default is 0).
        max_instances (int): The maximum number of instances (default is 4).
        idle_time_before_scale_down (int): Idle time in seconds before scaling down (default is 180).
        tier (str): The pricing tier, either "Dedicated" or "LowPriority" (default is "Dedicated").

    Returns:
        An instance of AmlCompute representing the compute resource.
    """
    try:
        # Check if the compute cluster already exists
        compute_resource = ml_client.compute.get(cluster_name)
        print(f"A cluster named '{cluster_name}' already exists; reusing it.")
    except Exception:
        print("Creating a new GPU compute resource...")
        compute_resource = AmlCompute(
            name=cluster_name,
            type="amlcompute",
            size=size,
            min_instances=min_instances,
            max_instances=max_instances,
            idle_time_before_scale_down=idle_time_before_scale_down,
            tier=tier,
        )
        # Create the compute resource and wait until the operation completes
        compute_resource = ml_client.begin_create_or_update(compute_resource).result()
    
    print(f"AMLCompute resource '{compute_resource.name}' is created with size '{compute_resource.size}'.")
    return compute_resource


In [ ]:
create_compute_resource(ml_client, 
                        training_gpu_cluster,
                        compute_name)

### What’s next?

* Subsequent training jobs can reference this cluster via  
  `compute=azureml:${{training_gpu_cluster}}`.  
* If you need **separate CPU and GPU pools**, repeat the helper with a different
  `cluster_name` and `size`.
* Cluster metadata (state, quota usage) is visible in **Azure ML Studio → Manage → Compute**.
